In [1]:
from sentence_transformers import SentenceTransformer, util
import bert_score
from rouge_score import rouge_scorer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
import numpy as np



c:\Users\zeina\Desktop\EventsEvaluation\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load models once
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

In [25]:
from nltk.translate.meteor_score import meteor_score
import nltk

nltk.download('wordnet')
    

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\zeina\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [36]:
# --- 1. Semantic Similarity (S-BERT cosine) ---
def compute_semantic_similarity_sbert(original_text: str, summary: str) -> float:
    embeddings = sbert_model.encode([original_text, summary], convert_to_tensor=True)
    similarity = util.pytorch_cos_sim(embeddings[0], embeddings[1]).item()
    return similarity


# --- 2. Content Similarity (BERTScore) ---
def compute_bertscore(original_text: str, summary: str) -> float:
    P, R, F1 = bert_score.score([summary], [original_text], lang='en', verbose=False)
    return F1[0].item()


# --- 3. Coverage (ROUGE-L) ---
def compute_rouge_l(original_text: str, summary: str) -> float:
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    score = scorer.score(original_text, summary)
    return score['rougeL'].fmeasure


# --- 4. Relevance ---
def relevance(original_text, summary):
    
    reference = [original_text.split()]
    hypothesis = summary.split()
    score = meteor_score(reference, hypothesis)
    return score

# --- 5. Relevance Spread ---

def relevance_spread(original_text, summary):
   
    sections = [original_text[i:i + len(original_text)//4] for i in range(0, len(original_text), len(original_text)//4)]
    meteor_scores = [meteor_score([s.split()], summary.split()) for s in sections if s]
    if not meteor_scores or len(meteor_scores) < 2:
        return 0
    std_dev = np.std(meteor_scores)
    mean_score = np.mean(meteor_scores)
    return float(std_dev / mean_score) if mean_score != 0 else 0


# --- Evaluation Wrapper ---
def evaluate_summary_v2(original_text: str, summary: str) -> dict:
    return {
        "Semantic Similarity (SBERT)": compute_semantic_similarity_sbert(original_text, summary),
        "Content Similarity (BERTScore)": compute_bertscore(original_text, summary),
        "Coverage (ROUGE-L)": compute_rouge_l(original_text, summary),
        "Relevance (METEOR)": relevance(original_text, summary),
        "Relevance Spread": relevance_spread(original_text, summary)
    }


In [4]:
original_text = """Once upon a time there was a dear little girl who was loved by every one who looked at her, but most of all by her grandmother, and there was nothing that she would not have given to the child. Once she gave her a little cap of red velvet, which suited her so well that she would never wear anything else. So she was always called Little Red Riding Hood.
One day her mother said to her, "Come, Little Red Riding Hood, here is a piece of cake and a bottle of wine. Take them to your grandmother, she is ill and weak, and they will do her good. Set out before it gets hot, and when you are going, walk nicely and quietly and do not run off the path, or you may fall and break the bottle, and then your grandmother will get nothing. And when you go into her room, don't forget to say, good-morning, and don't peep into every corner before you do it."
I will take great care, said Little Red Riding Hood to her mother, and gave her hand on it. The grandmother lived out in the wood, half a league from the village, and just as Little Red Riding Hood entered the wood, a wolf met her. Little Red Riding Hood did not know what a wicked creature he was, and was not at all afraid of him.
"Good-day, Little Red Riding Hood," said he.
"Thank you kindly, wolf."
"Whither away so early, Little Red Riding Hood?"
"To my grandmother's."
"What have you got in your apron?"
"Cake and wine. Yesterday was baking-day, so poor sick grandmother is to have something good, to make her stronger."
"Where does your grandmother live, Little Red Riding Hood?"
"A good quarter of a league farther on in the wood. Her house stands under the three large oak-trees, the nut-trees are just below. You surely must know it," replied Little Red Riding Hood.
The wolf thought to himself, "What a tender young creature. What a nice plump mouthful, she will be better to eat than the old woman. I must act craftily, so as to catch both." So he walked for a short time by the side of Little Red Riding Hood, and then he said, "see Little Red Riding Hood, how pretty the flowers are about here. Why do you not look round. I believe, too, that you do not hear how sweetly the little birds are singing. You walk gravely along as if you were going to school, while everything else out here in the wood is merry."
Little Red Riding Hood raised her eyes, and when she saw the sunbeams dancing here and there through the trees, and pretty flowers growing everywhere, she thought, suppose I take grandmother a fresh nosegay. That would please her too. It is so early in the day that I shall still get there in good time. And so she ran from the path into the wood to look for flowers. And whenever she had picked one, she fancied that she saw a still prettier one farther on, and ran after it, and so got deeper and deeper into the wood.
Meanwhile the wolf ran straight to the grandmother's house and knocked at the door.
"Who is there?"
"Little Red Riding Hood," replied the wolf. "She is bringing cake and wine. Open the door."
"Lift the latch," called out the grandmother, "I am too weak, and cannot get up."
The wolf lifted the latch, the door sprang open, and without saying a word he went straight to the grandmother's bed, and devoured her. Then he put on her clothes, dressed himself in her cap, laid himself in bed and drew the curtains.
Little Red Riding Hood, however, had been running about picking flowers, and when she had gathered so many that she could carry no more, she remembered her grandmother, and set out on the way to her.
She was surprised to find the cottage-door standing open, and when she went into the room, she had such a strange feeling that she said to herself, oh dear, how uneasy I feel to-day, and at other times I like being with grandmother so much.
She called out, "Good morning," but received no answer. So she went to the bed and drew back the curtains. There lay her grandmother with her cap pulled far over her face, and looking very strange.
"Oh, grandmother," she said, "what big ears you have."
"The better to hear you with, my child," was the reply.
"But, grandmother, what big eyes you have," she said.
"The better to see you with, my dear."
"But, grandmother, what large hands you have."
"The better to hug you with."
"Oh, but, grandmother, what a terrible big mouth you have."
"The better to eat you with."
And scarcely had the wolf said this, than with one bound he was out of bed and swallowed up Little Red Riding Hood.
When the wolf had appeased his appetite, he lay down again in the bed, fell asleep and began to snore very loud. The huntsman was just passing the house, and thought to himself, how the old woman is snoring. I must just see if she wants anything.
So he went into the room, and when he came to the bed, he saw that the wolf was lying in it. "Do I find you here, you old sinner," said he. "I have long sought you."
Then just as he was going to fire at him, it occurred to him that the wolf might have devoured the grandmother, and that she might still be saved, so he did not fire, but took a pair of scissors, and began to cut open the stomach of the sleeping wolf.
When he had made two snips, he saw the Little Red Riding Hood shining, and then he made two snips more, and the little girl sprang out, crying, "Ah, how frightened I have been. How dark it was inside the wolf."
And after that the aged grandmother came out alive also, but scarcely able to breathe. Little Red Riding Hood, however, quickly fetched great stones with which they filled the wolf's belly, and when he awoke, he wanted to run away, but the stones were so heavy that he collapsed at once, and fell dead.
Then all three were delighted. The huntsman drew off the wolf's skin and went home with it. The grandmother ate the cake and drank the wine which Little Red Riding Hood had brought, and revived, but Little Red Riding Hood thought to herself, as long as I live, I will never by myself leave the path, to run into the wood, when my mother has forbidden me to do so.
"""
summaryPrompts = """
A young girl with a basket walks through a forest, about halfway between the village and her destination.
A young girl with a basket picks flowers in a forest, unaware of danger lurking nearby.
A large wolf with sharp teeth pounces on a young girl in the forest.
A brave hunter enters a forest, where he finds a wolf with an unexpected surprise inside its belly.
A hunter uses a pair of scissors to cut into a wolf's belly, releasing two people from its innards.
A wolf with a stomach full of stones tries to run away, but ultimately succumbs to its own weight.
"""

summaryActions = """
A forest with tall trees, about a half-league from the village.
Little Red Riding Hood's mother gives her a piece of cake and a bottle of wine to bring to her grandmother.
Little Red Riding Hood deviates from the path to pick flowers and encounters a wolf.
The wolf tricks Little Red Riding Hood and eats her grandmother, then attempts to eat Little Red Riding Hood herself.
The huntsman arrives and discovers Little Red Riding Hood and her grandmother inside the wolf's stomach.
The huntsman cuts open the wolf's stomach and rescues Little Red Riding Hood and her grandmother.
The wolf awakens and tries to escape with the heavy stones in its stomach, but collapses and dies.
"""

In [37]:
evaluate_summary_v2(original_text, summaryPrompts.strip())

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'Semantic Similarity (SBERT)': 0.43760600686073303,
 'Content Similarity (BERTScore)': 0.8043063282966614,
 'Coverage (ROUGE-L)': 0.06725380304243395,
 'Relevance (METEOR)': 0.03690393424256991,
 'Relevance Spread': 0.09474727562626696}

In [38]:
evaluate_summary_v2(original_text, summaryActions.strip())

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'Semantic Similarity (SBERT)': 0.648106575012207,
 'Content Similarity (BERTScore)': 0.8321223855018616,
 'Coverage (ROUGE-L)': 0.12173913043478261,
 'Relevance (METEOR)': 0.06256928438851203,
 'Relevance Spread': 0.13997035067230038}

In [49]:
import re

def extract_prompts(text: str) -> list:
    # Matches 'Prompt: "..."' and captures the content inside the quotes
    pattern = r'Prompt:\s*"([^"]+)"'
    return re.findall(pattern, text)

def extract_actions(text: str) -> list:
    # Matches 'Actions: ...' and captures everything up to the next line starting with a capitalized field or 'Prompt:'
    pattern = r'Actions:\s*(.*?)(?=\n(?:Setting|Characters|Actions|Prompt|Event|\Z))'
    return [action.strip() for action in re.findall(pattern, text, flags=re.DOTALL)]


In [45]:
# Example usage:
input_text = """
Event 1:

Setting: A forest with tall trees, about a half-league from the village.
Characters: Little Red Riding Hood (key participant), her mother (not a key participant).
Actions: Little Red Riding Hood's mother gives her a piece of cake and a bottle of wine to bring to her grandmother.
Prompt: "A young girl with a basket walks through a forest, about halfway between the village and her destination."
Event 2:

Setting: The same forest as before.
Characters: Little Red Riding Hood (key participant).
Actions: Little Red Riding Hood deviates from the path to pick flowers and encounters a wolf.
Prompt: "A young girl with a basket picks flowers in a forest, unaware of danger lurking nearby."

Event 3:
Setting: The same forest as before.
Characters: Wolf (key participant), Little Red Riding Hood (key participant).
Actions: The wolf tricks Little Red Riding Hood and eats her grandmother, then attempts to eat Little Red Riding Hood herself.
Prompt: "A large wolf with sharp teeth pounces on a young girl in the forest."
Event 4:

Setting: The same forest as before.
Characters: Wolf (key participant), huntsman (key participant).
Actions: The huntsman arrives and discovers Little Red Riding Hood and her grandmother inside the wolf's stomach.
Prompt: "A brave hunter enters a forest, where he finds a wolf with an unexpected surprise inside its belly."
Event 5:

Setting: The same location as Event 4.
Characters: Huntsman (key participant), Little Red Riding Hood (key participant), grandmother (key participant).
Actions: The huntsman cuts open the wolf's stomach and rescues Little Red Riding Hood and her grandmother.
Prompt: "A hunter uses a pair of scissors to cut into a wolf's belly, releasing two people from its innards."
Event 6:

Setting: The same location as Event 5.
Characters: Wolf (key participant).
Actions: The wolf awakens and tries to escape with the heavy stones in its stomach, but collapses and dies.
Prompt: "A wolf with a stomach full of stones tries to run away, but ultimately succumbs to its own weight."
"""

prompts = extract_prompts(input_text)
for i, prompt in enumerate(prompts, 1):
    print(f"{prompt}")



A young girl with a basket walks through a forest, about halfway between the village and her destination.
A young girl with a basket picks flowers in a forest, unaware of danger lurking nearby.
A large wolf with sharp teeth pounces on a young girl in the forest.
A brave hunter enters a forest, where he finds a wolf with an unexpected surprise inside its belly.
A hunter uses a pair of scissors to cut into a wolf's belly, releasing two people from its innards.
A wolf with a stomach full of stones tries to run away, but ultimately succumbs to its own weight.


In [50]:
actions = extract_actions(input_text)
for i, action in enumerate(actions, 1):
    print(f"{action}")

Little Red Riding Hood's mother gives her a piece of cake and a bottle of wine to bring to her grandmother.
Little Red Riding Hood deviates from the path to pick flowers and encounters a wolf.
The wolf tricks Little Red Riding Hood and eats her grandmother, then attempts to eat Little Red Riding Hood herself.
The huntsman arrives and discovers Little Red Riding Hood and her grandmother inside the wolf's stomach.
The huntsman cuts open the wolf's stomach and rescues Little Red Riding Hood and her grandmother.
The wolf awakens and tries to escape with the heavy stones in its stomach, but collapses and dies.
